In [3]:
import pandas as pd
import numpy as np

from CNN_Audio_Model import CNN_Audio_Model
from TestDataset import TestDataset

import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl

import torchaudio
import os
import matplotlib.pyplot as plt
from pydub import AudioSegment

import os

/home/utworzyd/miniconda3/envs/pytorch_2/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [4]:
listFiles=[]

for filename in os.listdir("./Warblr/data/wav"):
    listFiles.append(filename)


In [5]:
#wczytanie wyniku z csv:
csv_path = "./warblrb10k_public_metadata.csv"
csv_data = pd.read_csv(csv_path)

In [8]:

class Classifier():
    def __init__(self,path, file_name):
        self.path = path
        self.file = file_name
        self.model = CNN_Audio_Model.load_from_checkpoint(checkpoint_path="./model_25e_vallist.ckpt")
        self.trainer = pl.Trainer()
#         self.model_prediction = self.run_model()
        self.model_prediction = self.run_model_with_resize()

    #TODO: to możnaby w sumie jakoś ładniej podzielić
    def run_model(self):
        #i tutaj jakoś sprawdzenie długości nagrania i ewentualne odpalenie kilka razy
        loadedAudio = AudioSegment.from_wav(self.path)

        time = loadedAudio.duration_seconds

        #dodanie ciszy, tak żeby długość nagrania była podzielna przez 10
        if time%10 !=0:
            silent_time = 10.00 - time%10
            silent_time = np.ceil(silent_time)
        else:
            silent_time = 0
        silent_segment = AudioSegment.silent(duration=silent_time*1000) 

        loadedAudio = loadedAudio + silent_segment
        counter = int(loadedAudio.duration_seconds/10)
        loadedAudio = loadedAudio[0:10*counter*1000]
        
        model_predictions = []
        for i in range(counter):
            cutAudio = loadedAudio[i*10*1000:10*(i+1)*1000]
            cutAudio.export('processedAudio.wav', format="wav")
            result = self.trainer.test(model=self.model,test_dataloaders = DataLoader(TestDataset( './processedAudio.wav', 1), batch_size = 64) )
            model_predictions.append( result[0]["test_epoch_end_accuracy"] )

        os.remove('./processedAudio.wav')
        if 1 in model_predictions:
            return 1
        else:
            return 0
        
    #run_model, w którym pliki są po prostu wczytywane, a problem zbyt dużego rozmiaru jest rozwiązywany przez resize
    def run_model_with_resize(self):
        result = self.trainer.test(model=self.model,test_dataloaders = DataLoader(TestDataset( self.path, 1), batch_size = 64) )
        return result[0]["test_epoch_end_accuracy"]



In [1]:
# warblrb10k_public_metadata.csv
goodPredictions = []
for file in listFiles:
    classif = Classifier("./Warblr/data/wav/"+file, file)
    #tu robisz troszkę bezsensowne przeszukiwanie drugi raz csvki
    csv_row = csv_data.loc[ csv_data['itemid']==file[:-4] ] 
    print(file)
    print(csv_row['hasbird'])
    data_prediction = int(csv_row['hasbird'] )
    if classif.model_prediction == data_prediction:
        goodPredictions.append(1)
        #goodPredictions = goodPredictions+1
    else:
        goodPredictions.append(0)
        
#zapis do pliku tego, co udało się uzyskać
with open('predictions_for_warblr.txt', 'w') as filehandle:
    for ide in range(len(goodPredictions)):
        filehandle.write('%s,' % listFiles[ide])
        filehandle.write('%s\n' % goodPredictions[ide])
        
print("Liczba dobrych predykcji: ",sum(goodPredictions))
print("Liczba wszystkich predykcji: ", len(goodPredictions))

NameError: name 'listFiles' is not defined